# Load pretrained pb

In [1]:
import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.python.framework import tensor_util

# path to your .pb file
GRAPH_PB_PATH = './v3-small_224_1.0_float/v3-small_224_1.0_float.pb'
with tf.Session() as sess:
    print("load graph")
    with gfile.FastGFile(GRAPH_PB_PATH, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        tf.import_graph_def(graph_def, name='')
        graph_nodes = [n for n in graph_def.node]

wts = [n for n in graph_nodes if n.op=='Const']
const_weights = []
for n in wts:
    const_weights.append((n.name, tensor_util.MakeNdarray(n.attr['value'].tensor)))

/Users/liew/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


load graph
Instructions for updating:
Use tf.gfile.GFile.


In [2]:
for name, weight in const_weights:
    print(name, weight.shape)
    

MobilenetV3/Conv/weights (3, 3, 3, 16)
MobilenetV3/Conv/BatchNorm/gamma (16,)
MobilenetV3/Conv/BatchNorm/beta (16,)
MobilenetV3/Conv/BatchNorm/moving_mean (16,)
MobilenetV3/Conv/BatchNorm/moving_variance (16,)
MobilenetV3/Conv/hard_swish/add/y ()
MobilenetV3/Conv/hard_swish/mul_1/y ()
MobilenetV3/expanded_conv/depthwise/depthwise_weights (3, 3, 16, 1)
MobilenetV3/expanded_conv/depthwise/BatchNorm/gamma (16,)
MobilenetV3/expanded_conv/depthwise/BatchNorm/beta (16,)
MobilenetV3/expanded_conv/depthwise/BatchNorm/moving_mean (16,)
MobilenetV3/expanded_conv/depthwise/BatchNorm/moving_variance (16,)
MobilenetV3/expanded_conv/squeeze_excite/Conv/weights (1, 1, 16, 8)
MobilenetV3/expanded_conv/squeeze_excite/Conv/biases (8,)
MobilenetV3/expanded_conv/squeeze_excite/Conv_1/weights (1, 1, 8, 16)
MobilenetV3/expanded_conv/squeeze_excite/Conv_1/biases (16,)
MobilenetV3/expanded_conv/squeeze_excite/Conv_1/add/y ()
MobilenetV3/expanded_conv/squeeze_excite/Conv_1/mul/y ()
MobilenetV3/expanded_conv/pr

MobilenetV3/expanded_conv_8/squeeze_excite/Conv_1/mul/y ()
MobilenetV3/expanded_conv_8/project/weights (1, 1, 288, 96)
MobilenetV3/expanded_conv_8/project/BatchNorm/gamma (96,)
MobilenetV3/expanded_conv_8/project/BatchNorm/beta (96,)
MobilenetV3/expanded_conv_8/project/BatchNorm/moving_mean (96,)
MobilenetV3/expanded_conv_8/project/BatchNorm/moving_variance (96,)
MobilenetV3/expanded_conv_9/expand/weights (1, 1, 96, 576)
MobilenetV3/expanded_conv_9/expand/BatchNorm/gamma (576,)
MobilenetV3/expanded_conv_9/expand/BatchNorm/beta (576,)
MobilenetV3/expanded_conv_9/expand/BatchNorm/moving_mean (576,)
MobilenetV3/expanded_conv_9/expand/BatchNorm/moving_variance (576,)
MobilenetV3/expanded_conv_9/expand/hard_swish/add/y ()
MobilenetV3/expanded_conv_9/expand/hard_swish/mul_1/y ()
MobilenetV3/expanded_conv_9/depthwise/depthwise_weights (5, 5, 576, 1)
MobilenetV3/expanded_conv_9/depthwise/BatchNorm/gamma (576,)
MobilenetV3/expanded_conv_9/depthwise/BatchNorm/beta (576,)
MobilenetV3/expanded_con

# Bulit MobileNetV3-Small

In [3]:
from keras.layers import Conv2D, DepthwiseConv2D, Dense, GlobalAveragePooling2D, AveragePooling2D
from keras.layers import Activation, BatchNormalization, Add, Multiply, Reshape

from keras import backend as K

K.set_learning_phase(0)


def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

class MobileNetBase:
    def __init__(self, shape, n_class, alpha=1.0):
        """Init
        
        # Arguments
            input_shape: An integer or tuple/list of 3 integers, shape
                of input tensor.
            n_class: Integer, number of classes.
            alpha: Integer, width multiplier.
        """
        self.shape = shape
        self.n_class = n_class
        self.alpha = alpha

#     def _relu6(self, x):
#         """Relu 6
#         """
#         return K.relu(x, max_value=6.0)

    def _hard_swish(self, x):
        """Hard swish
        """
        return x * K.relu(x + 3.0, max_value=6.0) / 6.0

    def _return_activation(self, x, nl):
        """Convolution Block
        This function defines a activation choice.
        # Arguments
            x: Tensor, input tensor of conv layer.
            nl: String, nonlinearity activation type.
        # Returns
            Output tensor.
        """
        if nl == 'HS':
            x = Activation(self._hard_swish)(x)
        if nl == 'RE':
            x = Activation('relu')(x)

        return x

    def _conv_block(self, inputs, filters, kernel, strides, nl,use_bias=False):
        """Convolution Block
        This function defines a 2D convolution operation with BN and activation.
        # Arguments
            inputs: Tensor, input tensor of conv layer.
            filters: Integer, the dimensionality of the output space.
            kernel: An integer or tuple/list of 2 integers, specifying the
                width and height of the 2D convolution window.
            strides: An integer or tuple/list of 2 integers,
                specifying the strides of the convolution along the width and height.
                Can be a single integer to specify the same value for
                all spatial dimensions.
            nl: String, nonlinearity activation type.
        # Returns
            Output tensor.
        """

        channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

        x = Conv2D(filters, kernel, padding='same', strides=strides,use_bias=use_bias)(inputs)
        x = BatchNormalization(axis=channel_axis)(x)

        return self._return_activation(x, nl)

    
    def _squeeze(self, inputs):
        """Squeeze and Excitation.
        This function defines a squeeze structure.
        # Arguments
            inputs: Tensor, input tensor of conv layer.
        """
        input_channels = int(inputs.shape[-1])
        squeeze_channels = _make_divisible(input_channels / 4, divisor=8)
        x = AveragePooling2D(pool_size=[int(inputs.shape[1]),int(inputs.shape[2])])(inputs)
        
#         torch.nn.AdaptiveAvgPool2d(output_size=(1, 1)),
        x = Conv2D(squeeze_channels, kernel_size=1, use_bias=True,activation='relu')(x)
        x = Conv2D(input_channels, kernel_size=1, use_bias=True,activation='hard_sigmoid')(x)
#         x = Reshape((1, 1, input_channels))(x)
        x = Multiply()([inputs, x])

        return x

    def _bottleneck(self, inputs, filters, kernel, e, s, squeeze, nl):
        """Bottleneck
        This function defines a basic bottleneck structure.
        # Arguments
            inputs: Tensor, input tensor of conv layer.
            filters: Integer, the dimensionality of the output space.
            kernel: An integer or tuple/list of 2 integers, specifying the
                width and height of the 2D convolution window.
            e: Integer, expansion factor.
                t is always applied to the input size.
            s: An integer or tuple/list of 2 integers,specifying the strides
                of the convolution along the width and height.Can be a single
                integer to specify the same value for all spatial dimensions.
            squeeze: Boolean, Whether to use the squeeze.
            nl: String, nonlinearity activation type.
        # Returns
            Output tensor.
        """

        channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
        input_shape = K.int_shape(inputs)

        tchannel = int(e)
        cchannel = int(self.alpha * filters)

        r = s == 1 and input_shape[3] == filters
        if e > filters:
            x = self._conv_block(inputs, tchannel, (1, 1), (1, 1), nl)
        else:
            x = inputs
        x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same', use_bias=False)(x)
        x = BatchNormalization(axis=channel_axis)(x)
        x = self._return_activation(x, nl)

        if squeeze:
            x = self._squeeze(x)

        x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding='same', use_bias=False)(x)
        x = BatchNormalization(axis=channel_axis)(x)

        if r:
            x = Add()([x, inputs])

        return x

    def build(self):
        pass


"""MobileNet v3 small models for Keras.
# Reference
    [Searching for MobileNetV3](https://arxiv.org/abs/1905.02244?context=cs)
"""


from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Reshape
from keras.utils.vis_utils import plot_model

# from model.mobilenet_base import MobileNetBase


class MobileNetV3_Small(MobileNetBase):
    def __init__(self, shape, n_class, alpha=1.0, include_top=True):
        """Init.
        # Arguments
            input_shape: An integer or tuple/list of 3 integers, shape
                of input tensor.
            n_class: Integer, number of classes.
            alpha: Integer, width multiplier.
            include_top: if inculde classification layer.
        # Returns
            MobileNetv3 model.
        """
        super(MobileNetV3_Small, self).__init__(shape, n_class, alpha)
        self.include_top = include_top

    def build(self, plot=False):
        """build MobileNetV3 Small.
        # Arguments
            plot: Boolean, weather to plot model.
        # Returns
            model: Model, model.
        """
        inputs = Input(shape=self.shape)

        x = self._conv_block(inputs, 16, (3, 3), strides=(2, 2), nl='HS',use_bias=False)

        x = self._bottleneck(x, 16, (3, 3), e=16, s=2, squeeze=True, nl='RE')
        x = self._bottleneck(x, 24, (3, 3), e=72, s=2, squeeze=False, nl='RE')
        x = self._bottleneck(x, 24, (3, 3), e=88, s=1, squeeze=False, nl='RE')
        x = self._bottleneck(x, 40, (5, 5), e=96, s=2, squeeze=True, nl='HS')
        x = self._bottleneck(x, 40, (5, 5), e=240, s=1, squeeze=True, nl='HS')
        x = self._bottleneck(x, 40, (5, 5), e=240, s=1, squeeze=True, nl='HS')
        x = self._bottleneck(x, 48, (5, 5), e=120, s=1, squeeze=True, nl='HS')
        x = self._bottleneck(x, 48, (5, 5), e=144, s=1, squeeze=True, nl='HS')
        x = self._bottleneck(x, 96, (5, 5), e=288, s=2, squeeze=True, nl='HS')
        x = self._bottleneck(x, 96, (5, 5), e=576, s=1, squeeze=True, nl='HS')
        x = self._bottleneck(x, 96, (5, 5), e=576, s=1, squeeze=True, nl='HS')

        x = self._conv_block(x, 576, (1, 1), strides=(1, 1), nl='HS')
        x = GlobalAveragePooling2D()(x)
        x = Reshape((1, 1, 576))(x)

        x = Conv2D(1024, (1, 1), padding='same')(x)
        x = self._return_activation(x, 'HS')

        if self.include_top:
            x = Conv2D(self.n_class, (1, 1), padding='same', activation='softmax')(x)
            x = Reshape((self.n_class,))(x)

        model = Model(inputs, x)

        if plot:
            plot_model(model, to_file='images/MobileNetv3_small.png', show_shapes=True)

        return model

Using TensorFlow backend.


In [4]:
K.clear_session()
model = MobileNetV3_Small(shape=(224,224,3), n_class=1001).build()

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 16) 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 16) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
depthwise_

# Load weights to every layer

In [7]:
j_cnt = 0
for i, l in enumerate(model.layers):
    if not len(l.weights):
        continue
#     print(i, l.name, len(l.weights))
#     print([w.shape for w in l.weights])
    
    layer_ws = []
    if 'batch_normalization' in l.name:
        layer_ws = [w for n,w in const_weights[j_cnt:j_cnt+4]]
        l.set_weights(layer_ws)
        j_cnt += 4
    elif 'conv2d' in  l.name:
        layer_ws = [w for n,w in const_weights[j_cnt:j_cnt+len(l.weights)]]
        l.set_weights(layer_ws)
        j_cnt += len(l.weights)
    else:
        print('skip', l.name, len(l.weights))
    
    while len(const_weights[j_cnt][1].shape) == 0:
        j_cnt += 1
    
    

# model.save_weights('weights.h5')

# Test image

In [20]:
import cv2

img = cv2.imread('cat.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224))
img = (img/127.5) - 1 

prob = model.predict(np.expand_dims(img,0))[0]

In [22]:
prob.argsort()[::-1][:5]

array([278, 275, 279, 272, 288])